The goal of this data is to help the retailer by using machine learning to make predictions about future sales based on the data provided.

In [72]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
set_config(display='diagram')

In [73]:
path = '/content/drive/MyDrive/Coding Dojo/sales_predictions.csv'
df = pd.read_csv(path)
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700


In [74]:
ml_df = df.copy()
# making a copy to test out our machine learning without changing our original dataframe

In [75]:
# Before splitting the data drop duplicates and fix inconsistencies in categorical data
ml_df.isna().sum().sum()

3873

In [76]:
ml_df.dropna(inplace=True)

In [77]:
ml_df.drop_duplicates(inplace=True)

In [78]:
# Identify the features (X) 
#and target (y): Assign the "Item_Outlet_Sales" column as your target 
#and the rest of the relevant variables as your features matrix.  
Target = 'Item_Outlet_Sales'
X = ml_df.drop(columns= Target)
y = ml_df[Target]

In [79]:
# Perform a train test split 
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
X_train.shape

(3487, 11)

In [80]:
# Create a preprocessing object to prepare the dataset for Machine Learning
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [81]:
# Make sure your imputation of missing values occurs after the train test split using SimpleImputer.  
num_columns = num_selector(X_train)
cat_columns = cat_selector(X_train)

In [82]:
X_train.isna().any()

Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [83]:
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='mean')

In [84]:
# create tuples of (imputer, selector) for each datatype
num_tuple = (median_imputer, num_selector)
cat_tuple = (freq_imputer, cat_selector)
# instantiate ColumnTransformer
col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder='passthrough')
col_transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('simpleimputer-1', SimpleImputer(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fcb88184e90>),
                                ('simpleimputer-2',
                                 SimpleImputer(strategy='most_frequent'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fcb88184ed0>)])

In [85]:
# fit ColumnTransformer on the training data
col_transformer.fit(X_train)
# transform both the training and testing data (this will output a NumPy array)
X_train_imputed = col_transformer.transform(X_train)
X_test_imputed = col_transformer.transform(X_test)
# change the result back to a dataframe
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_train_imputed.isna().any()

Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [89]:
X_train_ml_df = pd.DataFrame(X_train_processed)
X_train_ml_df.head(3)
#checking our machine learning 

,0,1,2,3,4,5,6,7,8,9,...,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595
0,0.743119,-0.712775,1.828109,1.327849,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.505876,-1.291052,0.603369,1.327849,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.119583,1.813319,0.244541,0.136187,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
